In [ ]:
!pip install -q PyMuPDF faiss-cpu sentence-transformers transformers


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
from google.colab import files
import fitz  # PyMuPDF
import numpy as np
import faiss

# Upload your PDF
uploaded = files.upload()
pdf_path = next(iter(uploaded))

# Step 1: Extract text
def extract_text_from_pdf(path):
    doc = fitz.open(path)
    return " ".join(page.get_text() for page in doc)

# Step 2: Chunk text
def split_text(text, max_tokens=100):
    sentences = text.split(". ")
    chunks, current = [], ""
    for sentence in sentences:
        if len((current + sentence).split()) < max_tokens:
            current += sentence + ". "
        else:
            chunks.append(current.strip())
            current = sentence + ". "
    if current:
        chunks.append(current.strip())
    return chunks

# Load PDF and prepare chunks
pdf_text = extract_text_from_pdf(pdf_path)
chunks = split_text(pdf_text)

# Step 3: Build FAISS index
embedder = SentenceTransformer("all-MiniLM-L6-v2")
embeddings = embedder.encode(chunks)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))

# Step 4: Load FLAN-T5 model
print("⏳ Loading model...")
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-base")
qa_pipeline = pipeline("text2text-generation", model=model, tokenizer=tokenizer)

# Step 5: Start Chat
chat_history = ""
print("\n🤖 Chatbot ready! Ask something from your PDF. Type 'exit' to stop.")
while True:
    user_input = input("You: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # Retrieve context from PDF
    query_embedding = embedder.encode([user_input])
    D, I = index.search(np.array(query_embedding), k=3)
    context = "\n".join([chunks[i] for i in I[0]])

    # Form question prompt
    prompt = f"Context: {context}\nQuestion: {user_input}"

    # Generate answer
    response = qa_pipeline(prompt, max_new_tokens=150)[0]['generated_text']
    print("Bot:", response)



Saving M_M_Arman_Hossain,.pdf to M_M_Arman_Hossain, (3).pdf
⏳ Loading model...


Device set to use cuda:0



🤖 Chatbot ready! Ask something from your PDF. Type 'exit' to stop.
You: Can you explain how your Exam Seat Planner System improves exam logistics at the British Council?


Token indices sequence length is longer than the specified maximum sequence length for this model (1003 > 512). Running this sequence through the model will result in indexing errors


Bot: It uses structured data (seat planner data, seat locators, and a seat planning layout) to organize and assign candidate seats within exam rooms. Users can upload the necessary data files, select a date and session, and generate a seating plan in just a few clicks.
You: What architecture did you use for the AI Finance Accountant Agent, and how does it handle voice commands?
Bot: RAG (Retrieval-Augmented Generation) architecture
You: You've worked with Django, FastAPI, and Flask. Which one do you prefer for API development?
Bot: Django
You: exit
